In [1]:
import geopandas as gpd

# Zonas Medellin

In [2]:
# List of areas to process
zo = ['aguacatala_HL', 'moravia_LH', 'floresta_HH']
# zo = ['moravia_LH']
# z = zo[0]
for z in zo:
    print(f"Procesando zona: {z}")

    #Load the edges and NDVI shapefiles
    edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
    nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
    medellin_ndvi = gpd.read_file(f'../data/input/geojson/medellin_ndvi/Medellin_ndvi_HexRes11.geojson')

    # Reproject to CRS EPSG:32618
    edges = edges.to_crs('epsg:32618')
    medellin_ndvi = medellin_ndvi.to_crs('epsg:32618')

    # Perform spatial intersection (spatial join) to obtain the intersection geometries
    interseccion = gpd.sjoin(edges, medellin_ndvi, how="inner", predicate="intersects")

    # Group by 'osmid' to summarize NDVI data for each edge
    gdf_medellin_ndvi_sum = interseccion.groupby('osmid').agg(
        num_medellin_ndvi=('ndvi', 'count'),   # Count how many hexagons intersect each edge
        sum_ndvi=('ndvi', 'sum')               # Sum the NDVI index values of intersecting hexagons
    ).reset_index()

    # Calculate the average NDVI index for each edge
    gdf_medellin_ndvi_sum['ind_ndvi'] = gdf_medellin_ndvi_sum['sum_ndvi'] / gdf_medellin_ndvi_sum['num_medellin_ndvi']

    # Merge the calculated NDVI index back to the edges GeoDataFrame
    edges = edges.merge(gdf_medellin_ndvi_sum[['osmid', 'ind_ndvi']], on='osmid', how='left')

    # Save the updated edges shapefile with NDVI data to the specified output path
    edges = edges.to_crs('epsg:4326')
    edges.to_file(f'../data/output/shape/physical_variables/vegetation/{z}/{z}_vegetation_edges.shp')
    nodes.to_file(f'../data/output/shape/physical_variables/vegetation/{z}/{z}_vegetation_nodes.shp')

    # Limit the study area to the neighborhood polygon
    edges = edges.reset_index()
    edges = edges.to_crs('epsg:32618')
    nodes = nodes.reset_index()
    nodes = nodes.to_crs('epsg:32618')

    if z == 'floresta_HH':
        w = 'Alto_Alta'
    elif z == 'moravia_LH':
        w = 'Bajo_Alta'
    elif z == 'aguacatala_HL':
        w = 'Alto_Baja'

    # Load polygon files
    polygon = gpd.read_file(f'../data/input/shape/Poligonos/{w}.shp')
    polygon = polygon.to_crs('epsg:32618')
    polygon = polygon['geometry'].union_all()
    polygon = gpd.GeoDataFrame(geometry = [polygon], crs = 'epsg:32618')

    # Select the segments that are inside the polygon
    for idx, row in polygon.iterrows():
        # Filter the edges contained in the polygon
        edges_in_polygon = edges[edges['geometry'].intersects(row['geometry'])]

    # List of the nodes inside the polygon based on the 'u' and 'v' columns of the edges inside the polygon
    nodes_in_polygon = edges_in_polygon['u'].to_list() + edges_in_polygon['v'].to_list()
    # Make the nodes unique and sort it
    nodes_in_polygon = list(set(nodes_in_polygon))
    nodes_in_polygon.sort()
    # Select the nodes inside the buffer
    nodes_in_polygon = nodes.set_index('osmid').loc[nodes_in_polygon].reset_index()

    # Export
    nodes_in_polygon = nodes_in_polygon.to_crs('epsg:4326')
    edges_in_polygon = edges_in_polygon.to_crs('epsg:4326')
    nodes_in_polygon.to_file(f'../data/output/shape/project_network_filter/{z}/{z}_vegetation_nodes.shp')
    edges_in_polygon.to_file(f'../data/output/shape/project_network_filter/{z}/{z}_vegetation_edges.shp')


Procesando zona: aguacatala_HL
Procesando zona: moravia_LH
Procesando zona: floresta_HH


# Medellin

In [3]:
z = 'medellin'

#Load the edges and NDVI shapefiles
edges = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_edges_proj_net_initial.shp')
nodes = gpd.read_file(f'../data/output/shape/project_network_initial/{z}/{z}_nodes_proj_net_initial.shp')
medellin_ndvi = gpd.read_file(f'../data/input/geojson/medellin_ndvi/Medellin_ndvi_HexRes11.geojson')

# Reproject to CRS EPSG:32618
edges = edges.to_crs('epsg:32618')
medellin_ndvi = medellin_ndvi.to_crs('epsg:32618')

# Perform spatial intersection (spatial join) to obtain the intersection geometries
interseccion = gpd.sjoin(edges, medellin_ndvi, how="inner", predicate="intersects")

# Group by 'osmid' to summarize NDVI data for each edge
gdf_medellin_ndvi_sum = interseccion.groupby('osmid').agg(
    num_medellin_ndvi=('ndvi', 'count'),   # Count how many hexagons intersect each edge
    sum_ndvi=('ndvi', 'sum')               # Sum the NDVI index values of intersecting hexagons
).reset_index()

# Calculate the average NDVI index for each edge
gdf_medellin_ndvi_sum['ind_ndvi'] = gdf_medellin_ndvi_sum['sum_ndvi'] / gdf_medellin_ndvi_sum['num_medellin_ndvi']

# Merge the calculated NDVI index back to the edges GeoDataFrame
edges = edges.merge(gdf_medellin_ndvi_sum[['osmid', 'ind_ndvi']], on='osmid', how='left')

# Save the updated edges shapefile with NDVI data to the specified output path
edges = edges.to_crs('epsg:4326')
edges.to_file(f'../data/output/shape/physical_variables/vegetation/{z}/{z}_vegetation_edges.shp')
nodes.to_file(f'../data/output/shape/physical_variables/vegetation/{z}/{z}_vegetation_nodes.shp')